In [1]:
# Modelos de regresión para dependencia espacial tipo CAR

import os
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import box
from rasterstats import zonal_stats
from libpysal.weights import Queen
from spreg import GMM_Error

ruta_geoformas = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Inventario_geoformas_karsticas_Dunita.shp"
ruta_contorno  = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Contorno_Dunita.shp"
ruta_dem       = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\Raster\dem_clip.tif"
ruta_slope     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\Raster\pendiente.tif"
ruta_twi       = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\Raster\TWI.tif"
ruta_cover     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Cobertura_final.shp"
ruta_drain     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Drenajes_clip_POT_final.shp"
ruta_fault     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Fallas_lineam.shp"

# Cargar datos vectoriales iniciales
gdf_points  = gpd.read_file(ruta_geoformas)
gdf_contour = gpd.read_file(ruta_contorno)

# Generar cuadrícula de 100x100 metros
cell_size = 100
minx, miny, maxx, maxy = gdf_contour.total_bounds
xs = np.arange(minx, maxx, cell_size)
ys = np.arange(miny, maxy, cell_size)
polygons = [box(x, y, x+cell_size, y+cell_size) for x in xs for y in ys]
grid = gpd.GeoDataFrame({'geometry': polygons}, crs=gdf_contour.crs)
grid = gpd.clip(grid, gdf_contour)

# Conteo de puntos (variable dependiente)
join = gpd.sjoin(grid, gdf_points, how='left', predicate='intersects')
counts = join.groupby(join.index).size()
grid['count'] = counts.reindex(grid.index).fillna(0).astype(int)

# Extraer estadísticas zonales de los rasters
grid['dem_mean']   = [s['mean'] if s and s['mean'] is not None else np.nan for s in zonal_stats(grid, ruta_dem,   stats=['mean'], nodata=-9999)]
grid['slope_mean'] = [s['mean'] if s and s['mean'] is not None else np.nan for s in zonal_stats(grid, ruta_slope, stats=['mean'], nodata=-9999)]
grid['twi_mean']   = [s['mean'] if s and s['mean'] is not None else np.nan for s in zonal_stats(grid, ruta_twi,   stats=['mean'], nodata=-9999)]

# Coberturas y calcular distancias
cov_gdf  = gpd.read_file(ruta_cover)[['d_N3_COBER','geometry']].to_crs(grid.crs)
centroids= grid.copy()
centroids['geometry'] = centroids.centroid
cov_join = gpd.sjoin(centroids, cov_gdf, how='left', predicate='within')
grid['cover'] = cov_join['d_N3_COBER'].fillna('None')

drains = gpd.read_file(ruta_drain).to_crs(grid.crs)
faults = gpd.read_file(ruta_fault).to_crs(grid.crs)

union_drains = drains.geometry.union_all()
union_faults = faults.geometry.union_all()

grid['dist_drain'] = centroids.geometry.distance(union_drains)
grid['dist_fault'] = centroids.geometry.distance(union_faults)

# Limpiar datos: eliminar filas con valores nulos para el modelo
grid_clean = grid.dropna().copy()

# Preparación de variables y matrices espaciales para el modelo CAR

# Variables continuas 
vars_continuas = ['dem_mean', 'slope_mean', 'twi_mean', 'dist_drain', 'dist_fault']

# Variables categóricas significativas (identificadas previamente)
vars_categoricas_sig = [
    'Bosque fragmentado',
    'Mosaico de cultivos y espacios naturales'
]
vars_coberturas_dummy = ['cov_Bosque fragmentado', 'cov_Mosaico de cultivos y espacios naturales']

# Crear variables dummy solo para las coberturas seleccionadas
df_car = grid_clean.copy()
df_dummies = pd.get_dummies(df_car['cover'], prefix='cov')
df_car = df_car.join(df_dummies[vars_coberturas_dummy])

# Lista final de variables explicativas (X)
X_vars = vars_continuas + vars_coberturas_dummy

# Preparar matrices para spreg
y = df_car['count'].values.reshape(-1, 1)
x = df_car[X_vars].astype(float).values

# Matriz de pesos espaciales (contigüidad tipo Queen)
w = Queen.from_dataframe(df_car)
w.transform = 'r' # Estandarizar por filas

# Ajuste del Modelo Autorregresivo Condicional (CAR) 

model_car = GMM_Error(
    y, x, w=w,
    name_y='count',
    name_x=X_vars,
    name_w='Queen'
)

print(model_car.summary)

C:\Users\esteb\AppData\Local\Temp\ipykernel_25624\1430344207.py:88: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(df_car)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: GM SPATIALLY WEIGHTED LEAST SQUARES (HET)
------------------------------------------------------------
Data set            :     unknown
Weights matrix      :       Queen
Dependent Variable  :       count                Number of Observations:        2426
Mean dependent var  :      1.4596                Number of Variables   :           8
S.D. dependent var  :      1.7774                Degrees of Freedom    :        2418
Pseudo R-squared    :      0.1011
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -0.78526         0.56962        -1.37857         0.16803
            dem_mean         0.00041         0.00020    